In [ ]:
#https://towardsdatascience.com/solving-geographic-travelling-salesman-problems-using-python-e57284b14cd7
#Todo:
#Open Google Earth, get latitude, longitude, and name of the place
#get the libraries info of the distance of those places,
#turn the matrix of the distances into a simetric matrix 
